<a href="https://colab.research.google.com/github/SWEN90006/tutorials/blob/main/SWEN900006_Tutorial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SWEN90006 Tutorial 2

## Introduction
The aim of this tutorial is twofold. First, it aims to give you some practise at deriving test cases from specifications. Second, it aims for you to start exploring the limits of your test cases, and of the specifications.

## The LWIG Program
**Input File:** The input file format is as follows. Each line saves
the data for a single student, which contains several fields.
Each field is separated by a colon.

Each line consists of the following fields, in order:

-   A student number, which must be a 5 digit, 6 digit or 9 digit
    number.

-   The student's month of birth, which must be a string of 3 alphabetic
    characters with the first character capitalised and the remaining in
    lower case. That is, it must be from the set:
    
    $$\{Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec\}$$

-   The day of birth, which must be a number from 1--31 and must be within
    the valid range of days in that month; that is, it cannot be 30
    February, because February never has 30 days.

-   The student's surname, which must be a string of alphabetic
    characters all in capitals.

-   The first letter of the student's first name, which must be a single
    capitalised alphabetic character.

-   The number of lectures that they slept through, which must be an
    integer between 0 and 24 (both inclusive).

If any input row is invalid, the program should print a warning message
and continue with the next record. If the program encounters a more
serious problem (e.g. unable to open input file), it will print an error
message and exit gracefully.


**Example 1**. *As an example, suppose we had the following data.*

-   *Student number: 12345*

-   *Month of Birth: May*

-   *Day of Birth: 26*

-   *Surname: CHAN*

-   *First letter of first name: K*

-   *Number of lecture(s) slept through: 0*

*The input line for this data would be `12345:May:26:CHAN:K:0`*

## Tasks

1. What is the input domain for the LWIG program? What are the input conditions for the LWIG program?

2. Derive input test-cases for the program using equivalence partitioning and boundary value analysis, draw a test template tree to derive equivalence classes systematically and without overlap. 

3. Implement your tests in the JUnit driver below. Do your tests find any faults?

4. We implemented 4 faults in the LWIG program, how many faults did your test reveal? Why does or doesn't equivalence partitioning find all of them? 

## Java Implementation

The following code is a minimal implementation of the LWIG program. 

For the purpose of this tutorial, let's assume that the input file has already been parsed into an array containing the important elements.

### Prepare the Java Kernel
Since Java is not natively supported by Colab, we need to run the following code to enable Java kernel on Colab.

1. Run the cell bellow (click it and press Shift+Enter),
2. Change the kernel to java_use (Runtime -> Change Runtime Type -> select **"java_use"** -> Save)
3. Try and run the following cells. The java kernel is ready if you can load the JUnit library in the following cell. 

### Trouble shooting
  * There are two Java runtimes having similar names that you can select in step 2. If you accidentally select **java** instead of **java_use**, Colab will enter an indefinite loop, show **connecting**, and the bottom bar has a message saying "Connecting to **java_tcp** Google Compute Engine backend", you should delete the runtime (Runtime -> Disconnect and Delete Runtime), and run step 1 again. 
    * The working runtime has a log message saying it is **connecting to java Google Compute Enginge Backend**

In [ ]:
%%sh
# Install java kernel
wget -q https://github.com/SpencerPark/IJava/releases/download/v1.3.0/ijava-1.3.0.zip 
unzip -q ijava-1.3.0.zip 
python install.py

# Install proxy for the java kernel
wget -qO- https://gist.github.com/wenta0g/67289a9b2e54b8128109abb3aff2194b/archive/0707f5d61d156ce2830505679994b0f1a606589b.tar.gz | tar xvz --strip-components=1
python install_ipc_proxy_kernel.py --kernel=java --implementation=ipc_proxy_kernel.py

In [ ]:
%%loadFromPOM

<dependency>
  <groupId>junit</groupId>
  <artifactId>junit</artifactId>
  <version>4.13.2</version>
</dependency>

## LWIG Implementation

The following is a basic Java implementation of LWIG.

In [ ]:
import java.time.*;
import java.time.format.*;
import java.util.Arrays;
import java.util.Collection;
import java.util.Locale;

public class LWIG {

    public static boolean isValidDate(String month, String date) {
        boolean isValid = false;
        try {
            DateTimeFormatter dtf = new DateTimeFormatterBuilder()
                .parseCaseInsensitive()         
                .appendPattern("d-MMM-uuuu")  
                .toFormatter(Locale.ENGLISH);
            LocalDate.parse(String.format("%s-%s-2012", date, month), dtf); // Hard coded a leap year or common year
            isValid = true;
        } catch (Exception e) {
            //e.printStackTrace();
            isValid = false;
        }
        return isValid;
    }
    
    public static boolean isValidID(String id) {
        String regEx = "^(\\d{5}|\\d{6}|\\d{9}|\\d{10})$"; // 
        return id.matches(regEx);
    }
    
    public static boolean isValidSurname(String name)
    {
        String regEx = "^[A-Z]+$";
        return name.matches(regEx);
    }
    
    public static boolean isValidFirstLetter(String name)
    {
        String regEx = "^[A-Z]$";
        return name.matches(regEx);
    }
    
    public static boolean isValidSleptCount(String number)
    {
        boolean isValid = false;
        try {
            int count = Integer.parseInt(number);
            if (count >= 0 && count < 24) {
                isValid = true;
            }
        } catch (Exception e) {
            isValid = false;
        }
        return isValid;
    }
}

##JUnit test script

The following code block is a JUnit test script. JUnit is a unit-testing framework for Java that allows you to easily create tests that can be run automatically. 

In the code block below, put your test cases where is says "Your test cases start here". Add test cases by adding new elements to the data array. These will then be executed automatically by JUnit.

In [ ]:
import junit.framework.TestCase;

import org.junit.Test;
import org.junit.runner.*;
import org.junit.runner.RunWith;
import org.junit.runner.notification.Failure;
import org.junit.runners.Parameterized;

@RunWith(Parameterized.class)
public class LWIGTestCase extends TestCase {
    
    @Parameterized.Parameter(0)
    public String id;
    @Parameterized.Parameter(1)
    public String month;
    @Parameterized.Parameter(2)
    public String date;
    @Parameterized.Parameter(3)
    public String surname;
    @Parameterized.Parameter(4)
    public String firstLetter;
    @Parameterized.Parameter(5)
    public String sleptCount;
    @Parameterized.Parameter(6)
    public boolean result;
    
    @Test
    public void testLWIG() {
        boolean expectedResult = LWIG.isValidID(id) &&
            LWIG.isValidDate(month, date) &&
            LWIG.isValidSurname(surname) &&
            LWIG.isValidFirstLetter(firstLetter) &&
            LWIG.isValidSleptCount(sleptCount);
        assertEquals(result, expectedResult);
    }

    @Parameterized.Parameters(name = "Test case {index} failed: LWIG with id = {0}, Mon = {1}, date = {2}, surname = {3}, first letter = {4}, slept count = {5}, expected result = {6}")
    public static Collection<Object[]> data() {
        Object[][] data = new Object[][]{
            // Your Test cases start here
            {"13456", "Jan", "30", "CHAN", "K", "1", true}, 
            {"12345678", "Feb", "26", "TOM", "3", "0", true}
            // Your Test cases end here
            };
        return Arrays.asList(data);
    }
}

In [ ]:
Result result = JUnitCore.runClasses(LWIGTestCase.class);
for (Failure failure : result.getFailures()) {
     System.out.println(failure.toString());
}
System.out.println(String.format("Total run count: %s, Failed run count: %s", result.getRunCount(), result.getFailureCount()));